In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn
import seaborn as sns
from matplotlib.font_manager import FontProperties
from matplotlib.patches import Rectangle
from sklearn.metrics import balanced_accuracy_score
from sklearn.preprocessing import LabelEncoder

from classes.classes import ColorManager
from classes.classes import FlatModelActCardModelConfig, ZeroShotModelActCardConfig, DACEModelActCardConfig, \
    QPPModelActCardsConfig
from classes.classes import MODEL_CONFIGS
from classes.classes import ScaledPostgresModelConfig, FlatModelConfig, E2EModelConfig, ZeroShotModelConfig, \
    DACEModelConfig, QueryFormerModelConfig
from classes.paths import LocalPaths
from classes.workloads import EvalWorkloads
from evaluation.plots.utils import get_model_results

STANDARD_HATCHES = ['//', '\\\\', None,  '+++', '..', '--', 'xx', 'o', '///']

sns.set_theme(style="whitegrid", font_scale=1)
fontsize = 14

In [ ]:
# General settings
path = LocalPaths().data / "plots" / "scan_costs_anecdote.pdf"
indexes = [0, 9]
titles = ["High Selectivity \n(production_year >= 1880)", "Low Selectivity\n(production_year >= 2011)"]
palette = ColorManager.COLOR_PALETTE
hatch_patterns = {"SeqScan": "---", "IndexScan": ".."}
sort_map = {model.name.DISPLAY_NAME: i for i, model in enumerate(MODEL_CONFIGS)}
sort_map["Real Runtime"] = -2
sort_map[" "] = -1
model_configs = MODEL_CONFIGS
checkmark_font = FontProperties(family='DejaVu Sans', size=18)

# Generate plot with two subplots
fig, axs = plt.subplots(1, 2, figsize=(7.5, 1.5), dpi=100, sharey=False)

# Readout results for the two queries
seq_results = get_model_results(EvalWorkloads.ScanCostsPercentile.imdb_seq[-1], model_configs)
index_results = get_model_results(EvalWorkloads.ScanCostsPercentile.imdb_idx[-1], model_configs)

for index, title, prediction_ax in zip(indexes, titles, axs):
    # Readout results for the two queries
    seq_res = seq_results[seq_results['query_index'] == index].assign(query_index="SeqScan")
    idx_res = index_results[index_results['query_index'] == index].assign(query_index="IndexScan")
    results = pd.concat([seq_res, idx_res], ignore_index=True)
    
    # Add a new model that displays the real runtime
    real_runtime = results.groupby("query_index")["label"].min().reset_index()
    real_runtime["model"] = "Real Runtime"
    real_runtime["prediction"] = real_runtime["label"]
    real_runtime["label"] = real_runtime["label"]
    real_runtime["runtime"] = real_runtime["label"]
    real_runtime["selected_runtime"] = real_runtime["runtime"]
    results = pd.concat([results, real_runtime], ignore_index=True)
    
    # Add empty model to generate a gap between the two groups
    results = pd.concat([results.iloc[:0], pd.DataFrame({'model': [' '], 'query_index': ['IndexScan'], 'prediction': 0}), results.iloc[0:]])
    results = pd.concat([results.iloc[:0], pd.DataFrame({'model': [' '], 'query_index': ['SeqScan'], 'prediction': 0}), results.iloc[0:]])
    
    # Sort the results
    results = results.sort_values(by='model', key=lambda x: x.map(sort_map))    
    results = results.sort_values(by=['query_index', 'model'], key=lambda x: x.map({"SeqScan": 0, "IndexScan": 1}))
    
    # Draw predictions
    bars = seaborn.barplot(x="model",
                    hue="query_index",
                    y="prediction",
                    data=results,
                    ax=prediction_ax,
                    errorbar=None,
                    edgecolor='black',
                    width=0.85)
        
    # Add symbols to the top for the model selection
    if index == 0:
        symbols = ['✓', '✓', '⨯', '⨯', '✓', '⨯', '⨯', '⨯', '✓']
    else:
        symbols = ['⨯', '⨯', '✓', '⨯', '✓', '✓', '✓', '✓', '⨯']
        
    for (x, symbol) in zip(range(1, 1 + len(MODEL_CONFIGS)), symbols):
        if symbol == '✓':
            color = 'green'
        else:
            color = 'red'
        prediction_ax.annotate(xy=(0.093 * (x + 0.9), 1.02), xycoords='axes fraction', text=symbol, rotation=0, fontproperties=checkmark_font, color=color, fontsize=fontsize * 1.3)
    
    # Group bars together
    bar_groups = []
    for bar, label in zip(bars.patches, bars.get_xticklabels() + bars.get_xticklabels()):
        row = {"x": bar.get_x(), "model":  label.get_text(), "bar": bar}
        bar_groups.append(row)
    
    # Sort bar groups by x position
    bar_groups = sorted(bar_groups, key=lambda x: x['x'])
    
    # Set background color for each group of bars (grouped by model)
    for bar_group in bar_groups:
        color = ColorManager.COLOR_PALETTE.get(bar_group["model"], 'white')
        bar_group["bar"].set_facecolor(color)
        
    # Apply hatch patterns
    for bar, workload_type in zip(bars.patches, results['query_index']):
        bar.set_hatch(hatch_patterns[workload_type])

    # Plot settings
    prediction_ax.set_xlabel("")
    prediction_ax.set_yscale('log')
    prediction_ax.grid(axis="y", which='both', linestyle='--', linewidth=0.5)
    prediction_ax.tick_params(axis='x', rotation=45, labelsize=fontsize * 0.8, pad=-4)
    prediction_ax.tick_params(axis='y', rotation=0, labelsize=fontsize * 0.8,)
    prediction_ax.get_legend().remove()
    prediction_ax.set_xticklabels(["Real Runtime", ""] + [m.name.DISPLAY_NAME for m in MODEL_CONFIGS], fontsize=fontsize * 0.8, rotation=45, ha='right')
    prediction_ax.set_title(label=title, fontsize=fontsize, y=1.2, horizontalalignment='center')
    
    # Create offset transform by 5 points in x direction
    dx = 8/72.; dy = 0/72. 
    import matplotlib
    offset = matplotlib.transforms.ScaledTranslation(dx, dy, fig.dpi_scale_trans)
    
    # apply offset transform to all x ticklabels.
    for label in prediction_ax.xaxis.get_majorticklabels():
        label.set_transform(label.get_transform() + offset)
        
    
    if index == 0:
        prediction_ax.set_ylim(1e-2, 10)
        prediction_ax.set_ylabel("Predicted Runtime (s)", fontsize=fontsize * 0.8)
        
    elif index == 9:
        prediction_ax.set_ylim(1e-5, 10)
        prediction_ax.set_ylabel("")
        
    # Add a letter to the top left corner    
    letter = "A" if index == 0 else "B"
    prediction_ax.annotate(
            letter,
            xy=(0.18, 1.50), 
            xycoords='axes fraction', 
            fontsize=12, 
            ha='center', 
            va='center', 
            bbox=dict(boxstyle='circle,pad=0.2', edgecolor='black', facecolor='white'))

    # Set legend
    if index == 0:
        handles = [Rectangle((0, 0), 1, 1, facecolor='white', edgecolor='black', label='Seq Scan', hatch=hatch_patterns["SeqScan"]),
                   Rectangle((0, 0), 1, 1, facecolor='white', edgecolor='black', label='Index Scan', hatch=hatch_patterns["IndexScan"])]
        legend = prediction_ax.legend(handles=handles, 
                                      fontsize=fontsize,
                                      ncol=2,
                                      loc='center left',
                                      bbox_to_anchor=(0.5, -0.9),
                                      labelspacing=0.0,
                                      columnspacing=0.8,
                                      edgecolor='white')
plt.subplots_adjust(wspace=0.18)
fig.align_labels()
fig.savefig(path, bbox_inches='tight')

### Plots over selectivity range

In [ ]:
label_encoder = LabelEncoder()
path = LocalPaths().data / "plots" / "scan_costs_over_column.pdf"
mosaic = """AAAABB\nAAAACC\nAAAADD"""
hatches = ['///', '-|||', '\\\\', '+++', '...', '\\\\--', 'xx', '---']

seq_workloads, idx_workloads = EvalWorkloads.ScanCostsPercentile.imdb_seq, EvalWorkloads.ScanCostsPercentile.imdb_idx

acc_dataframes = []

for seq_workload, idx_workload in zip(seq_workloads, idx_workloads):
    # Get and rename results
    seq_results = get_model_results(seq_workload, model_configs).rename(columns={'prediction': 'prediction_seq', 'label': 'label_seq'})
    idx_results = get_model_results(idx_workload, model_configs).rename(columns={'prediction': 'prediction_idx', 'label': 'label_idx'})
    results = pd.concat([seq_results, idx_results[['prediction_idx', 'label_idx']]], axis=1)
    
    # New: Remove slightly deviating query labels:
    results['label_idx'] = results.groupby('query_index')['label_idx'].transform('last')
    results['label_seq'] = results.groupby('query_index')['label_seq'].transform('last')
    
    # Assign optimal and predicted operators
    results = results.assign(
        optimal_operator=np.where(results['label_idx'] < results['label_seq'], 'idx', 'seq'),
        predicted_operator=np.where(results['prediction_idx'] < results['prediction_seq'], 'idx', 'seq'))
    
    # Give the index of the query where the label difference between idx and seq is minimal
    min_diff_idx = (results['label_idx'] - results['label_seq']).abs().idxmin()
    max_label = max(results['label_idx'].max(), results['label_seq'].max())

    # Create selected_runtime column
    results['selected_runtime'] = np.where(results['prediction_idx'] < results['prediction_seq'], results['label_idx'], results['label_seq'])
    
    # Scale act_card and calculate accuracy
    results['act_card'] = (results['act_card'] - results['act_card'].min()) / (results['act_card'].max() - results['act_card'].min())
    #results['accuracy'] = (results['optimal_operator'] == results['predicted_operator']).astype(int)
    #results['accuracy'] = balanced_accuracy_score(results['optimal_operator'], results['predicted_operator'])
    results.loc[:, 'predicted_operator_encoded'] = label_encoder.fit_transform(results['predicted_operator'])
    results.loc[:, 'optimal_operator_encoded'] = label_encoder.fit_transform(results['optimal_operator'])

    fig, axs = plt.subplots(1, len(model_configs) + 2, figsize=(14, 1.5), dpi=100, sharey=False)
    x_index = "act_card"
        
    real_value_ax = axs[0]
    prediction_axs = axs[1:-1]
    accuracy_ax = axs[-1]
    balanced_accuracy_scores = []
    f1_scores = []
    
    for model, prediction_ax in zip(model_configs, prediction_axs):
        model_results = results[results['model'] == model.name.DISPLAY_NAME]
        
        # Delete rows that contain NaN values (which happens for timeouts)
        model_results = model_results.dropna()
    
        # Calculate balanced accuracy        
        accuracy = balanced_accuracy_score(model_results["optimal_operator"], model_results["predicted_operator"])
        balanced_accuracy_scores.append((model.name.DISPLAY_NAME, accuracy))
        
        # Plot optimal sequential runtimes
        seaborn.lineplot(x=x_index,
                         y="label_seq",
                         linestyle="solid",
                         ax=prediction_ax,
                         data=model_results[model_results.index < min_diff_idx],
                         label="Seq Runtime",
                         color="green",
                         #linewidth=2,
                         zorder=100)
        
        # Plot optimal index runtimes
        seaborn.lineplot(x=x_index,
                         y="label_idx",
                         linestyle="dotted",
                         ax=prediction_ax,
                         data=model_results[model_results.index >= min_diff_idx],
                         label="Index Runtime",
                         color="green",
                         #linewidth=2,
                         zorder=100)
        
        # Plot selected runtimes from the model
        seaborn.scatterplot(x=x_index,
                         y="selected_runtime",
                         data=model_results,
                         ax=prediction_ax,
                         label=model.name.DISPLAY_NAME,
                         color=model.color(),
                         s=60,
                         linewidth=1,
                         marker="X",
                         zorder=110)
        
        seaborn.lineplot(x=x_index, y="label_idx", linestyle="dashed", ax=prediction_ax, data=results[results.query_index >= min_diff_idx], label=None, color="green", zorder=100, linewidth=3)
        seaborn.lineplot(x=x_index, y="label_idx", linestyle="dashed", ax=prediction_ax, data=results[results.query_index <= min_diff_idx], label=None, color="black", zorder=100, linewidth=3)
        seaborn.lineplot(x=x_index, y="label_seq", linestyle="solid", ax=prediction_ax, data=results[results.query_index <= min_diff_idx], label=None, color="green",zorder=100, linewidth=3)
        seaborn.lineplot(x=x_index, y="label_seq", linestyle="solid", ax=prediction_ax, data=results[results.query_index >= min_diff_idx], label=None, color="black",  zorder=100, linewidth=3)
    
        
        # Plot settings
        prediction_ax.set_xlabel("")
        prediction_ax.set_ylabel("")
        prediction_ax.tick_params(axis='x', rotation=0, labelsize=fontsize * 0.8)
        prediction_ax.tick_params(axis='y', rotation=0, labelsize=fontsize * 0.8)
        prediction_ax.set_ylim(max_label * -0.1 , max_label * 1.1)
        prediction_ax.set_xlim(-0.1, 1.1)
        prediction_ax.set_title(model.name.DISPLAY_NAME, fontsize=fontsize * 0.9, backgroundcolor=model.color(), y=1.05, color="white")
        prediction_ax.set_xticks([0, 0.5, 1], labels=[0, "", 1])
        prediction_ax.set_yticklabels([])
        prediction_ax.grid(axis="y", which='both', linestyle='--', linewidth=0.5)
        prediction_ax.get_legend().remove()
    

    # Plot accuracies
    accuracy_df = pd.DataFrame(balanced_accuracy_scores, columns=['model', 'accuracy'])
    seaborn.barplot(x="model",
                    y="accuracy",
                    hue="model",
                    data=accuracy_df,
                    ax=accuracy_ax,
                    palette=ColorManager.COLOR_PALETTE,
                    errorbar=None,
                    order=[model.name.DISPLAY_NAME for model in model_configs],
                    width=1,
                    edgecolor='black')
    
    accuracy_ax.set_xlabel("")
    accuracy_ax.set_xticks([])    
    accuracy_ax.set_ylabel("Accuracy", fontsize=fontsize * 0.8)
    accuracy_ax.set_ylim(0, 1)
    accuracy_ax.yaxis.tick_right()
    accuracy_ax.tick_params(axis='x', rotation=0, labelsize=fontsize * 0.8, size=0)
    accuracy_ax.tick_params(axis='y', rotation=0, labelsize=fontsize * 0.8)
    accuracy_ax.set_ylabel("")
    accuracy_ax.set_title("Balanced\nAccuracy", fontsize=fontsize * 0.8, backgroundcolor='white', y=1.05)
    
    # Add hatch patterns
    for bar, hatch in zip(accuracy_ax.patches, STANDARD_HATCHES):
        bar.set_hatch(hatch)

    # Add selectivity label        
    prediction_axs[3].set_xlabel("Selectivity", fontsize=fontsize * 0.8)
    seaborn.lineplot(x=x_index, y="label_idx", linestyle="dashed", ax=real_value_ax, data=results[results.query_index >= min_diff_idx], label=None, color="green", zorder=100, linewidth=3)
    seaborn.lineplot(x=x_index, y="label_idx", linestyle="dashed", ax=real_value_ax, data=results[results.query_index <= min_diff_idx], label="Index Scan", color="black", zorder=100, linewidth=3)
    seaborn.lineplot(x=x_index, y="label_seq", linestyle="solid", ax=real_value_ax, data=results[results.query_index <= min_diff_idx], label=None, color="green",zorder=110, linewidth=3)
    seaborn.lineplot(x=x_index, y="label_seq", linestyle="solid", ax=real_value_ax, data=results[results.query_index >= min_diff_idx], label="Seq Scan", color="black",  zorder=100, linewidth=3)
    
    real_value_ax.set_xlabel("", fontsize=fontsize)
    real_value_ax.set_xticks([0, 0.5, 1], labels=[0, "", 1])
    real_value_ax.set_xlim(-0.1,   1.1)
    real_value_ax.set_ylim(max_label * -0.1,  max_label * 1.1)
    real_value_ax.set_ylabel("Real/Selected\nRuntime (s)", fontsize=fontsize * 0.8)
    real_value_ax.tick_params(axis='x', rotation=0, labelsize=fontsize * 0.8)
    real_value_ax.tick_params(axis='y', rotation=0, labelsize=fontsize * 0.8)
    real_value_ax.set_title("Real Value", fontsize=fontsize * 0.8, backgroundcolor='white', y=1.05)
    real_value_ax.annotate('Optimal\nSelection', xy=(0.3, 0.01), xytext=(0.3, 0.01), fontsize=fontsize * 0.7, color='green')
    real_value_ax.legend(fontsize=fontsize * 0.9, ncol=1, loc='upper left', bbox_to_anchor=(-3, 0.7), labelspacing=0.3, edgecolor='white')
    
    fig.align_labels()
    plt.subplots_adjust(wspace=0.2)
    if seq_workload.get_workload_name() == "seq.title.production_year":
        # Print the selectivity of the query where the difference is minimal
        print(seq_workload.get_workload_name())
        print(f"Breaking point at: {results.loc[min_diff_idx, 'act_card'] / max(results['act_card'])}")
        fig.savefig(path, bbox_inches='tight')
        plt.show()
    plt.close()

### Results over multiple workloads and columns

In [ ]:
workload_pairs = [
    (EvalWorkloads.ScanCostsPercentile.imdb_seq, EvalWorkloads.ScanCostsPercentile.imdb_idx),
    (EvalWorkloads.ScanCostsPercentile.baseball_seq, EvalWorkloads.ScanCostsPercentile.baseball_idx),
    (EvalWorkloads.ScanCostsPercentile.tpc_h_pk_seq, EvalWorkloads.ScanCostsPercentile.tpc_h_pk_idx)]

accuracies = []

for (seq, idx) in workload_pairs:
    for (seq_workload, idx_workload) in zip(seq, idx):
        seq_results = get_model_results(seq_workload, model_configs).rename(columns={'prediction': 'prediction_seq', 'label': 'label_seq'})
        idx_results = get_model_results(idx_workload, model_configs).rename(columns={'prediction': 'prediction_idx', 'label': 'label_idx'})
        results = pd.concat([seq_results, idx_results[['prediction_idx', 'label_idx']]], axis=1)

        # New: Remove slightly deviating query labels:
        results['label_idx'] = results.groupby('query_index')['label_idx'].transform('first')
        results['label_seq'] = results.groupby('query_index')['label_seq'].transform('first')

        results = results.assign(
            optimal_operator=np.where(results['label_idx'] < results['label_seq'], 'idx', 'seq'),
            predicted_operator=np.where(results['prediction_idx'] < results['prediction_seq'], 'idx', 'seq'))

        # Give the index of the query where the label difference between idx and seq is minimal
        min_diff_idx = (results['label_idx'] - results['label_seq']).abs().idxmin()

        # Compute q-error of cardinalities as well:
        results['q_error'] = results.apply(lambda row: max(row['act_card'], row['est_card']) / min(row['act_card'], row['est_card']) if row['est_card'] != 0 and row['act_card'] != 0 else np.nan, axis=1)

        # Scale act_card and calculate accuracy
        results['act_card'] = (results['act_card'] - results['act_card'].min()) / (results['act_card'].max() - results['act_card'].min())
        results['accuracy'] = (results['optimal_operator'] == results['predicted_operator']).astype(int)
        results['accuracy'] = balanced_accuracy_score(results['optimal_operator'], results['predicted_operator'])

        # Calculate the accuracy for each model
        accuracy = results.groupby("model")["accuracy"].mean()

        balanced_accuracy_scores = []
        for model in model_configs:
            model_results = results[results['model'] == model.name.DISPLAY_NAME]

            # Delete rows that contain NaN values (which happens for timeouts)
            model_results = model_results.dropna()

            accuracy = round(balanced_accuracy_score(model_results["optimal_operator"], model_results["predicted_operator"]), 2)
            balanced_accuracy_scores.append((model.name.DISPLAY_NAME, accuracy))
            #f1_sc = round(f1_score(model_results["optimal_operator"], model_results["predicted_operator"], average='weighted'), 2)
            #f1_scores.append((model.name.DISPLAY_NAME, f1_sc))

        accuracy_df = pd.DataFrame(balanced_accuracy_scores, columns=['model', 'f1']).T
        accuracy_df.columns = accuracy_df.iloc[0]
        accuracy_df.drop(accuracy_df.index[0], inplace = True)
        accuracy_df['table'] = seq_workload.table
        accuracy_df['column'] = seq_workload.column
        accuracy_df['workload'] = seq_workload.database.db_name
        accuracy_df["q_error"] = results["q_error"].median()
        accuracies.append(accuracy_df)

result = pd.concat(accuracies, axis=0)
result = result.round(2)
# Add new row for each dataset with the main accuracy

result.reset_index(drop=True, inplace=True)
result = result[["workload", "table", "column", "Sc. PG10", "Sc. PG16", "Flat Vector", "MSCN", "E2E", "Zero-Shot", "QPP-Net", "QueryFormer", "DACE", "q_error"]]
result["table"] = result["table"].str.replace("_", "\_")
result["column"] = result["column"].str.replace("_", "\_")
average =  result[["workload", "Sc. PG10", "Sc. PG16", "Flat Vector", "MSCN", "E2E", "Zero-Shot", "QPP-Net", "QueryFormer","DACE", "q_error",]].groupby("workload").mean()
average = average.astype('float64').round(2)
average["workload"] = average.index
average["table"] = None
average["column"] = None
# concat average to results
result = pd.concat([result, average], axis=0)
result.sort_values(by=["workload", "table", "column"], inplace=True)
result

In [ ]:
total_average = result[["Sc. PG10", "Sc. PG16", "Flat Vector", "MSCN", "E2E", "Zero-Shot", "QPP-Net", "QueryFormer","DACE"]].mean()
total_average

### Preference over selectivity

In [ ]:
from classes.classes import ScaledPostgresModelV16Config

act_card_model_configs = [
    ScaledPostgresModelConfig(),
    ScaledPostgresModelV16Config(),
    FlatModelActCardModelConfig(),
    QPPModelActCardsConfig(),
    ZeroShotModelActCardConfig(),
    DACEModelActCardConfig(),
]

path = LocalPaths().data / "plots" / "scan_preference.pdf"
seq_workloads = EvalWorkloads.ScanCostsPercentile.imdb_seq + EvalWorkloads.ScanCostsPercentile.tpc_h_pk_seq + EvalWorkloads.ScanCostsPercentile.baseball_seq
idx_workloads = EvalWorkloads.ScanCostsPercentile.imdb_idx + EvalWorkloads.ScanCostsPercentile.tpc_h_pk_idx + EvalWorkloads.ScanCostsPercentile.baseball_idx

fig, axs = plt.subplots(1, 2, figsize=(8, 3), dpi=100, sharey=True)
all_markers = [['^', 'D', 's', 'X', 'd', 'H', 'p', 'v', '<'], ['^', 's', 'd', 'p', 'v', '<']]

for model_conf, ax, markers in zip([model_configs, act_card_model_configs], axs, all_markers):
    all_results = []
    for seq_workload, idx_workload in zip(seq_workloads, idx_workloads):
        # Get and rename results
        seq_results = get_model_results(seq_workload, model_conf).rename(
            columns={'prediction': 'prediction_seq', 'label': 'label_seq'})
        idx_results = get_model_results(idx_workload, model_conf).rename(
            columns={'prediction': 'prediction_idx', 'label': 'label_idx'})
        results = pd.concat([seq_results, idx_results[['prediction_idx', 'label_idx']]], axis=1)
        # Assign optimal and predicted operators
        results = results.assign(
            optimal_operator=np.where(results['label_idx'] < results['label_seq'], 'idx', 'seq'),
            predicted_operator=np.where(results['prediction_idx'] < results['prediction_seq'], 'idx', 'seq'))

        # Give the index of the query where the label difference between idx and seq is minimal
        min_diff_idx = (results['label_idx'] - results['label_seq']).abs().idxmin()
        # Normalize the cardinalities. Use rank method here. This is not fully precise, as act_card can be in between of 0.1 steps
        results['normalized_card'] = (results['act_card'].rank(method='dense') - 1) / 10
        results['workload'] = seq_workload.get_workload_name()
        results['accuracy'] = (results['optimal_operator'] == results['predicted_operator']).astype(int)
        all_results.append(results)

    total_results = pd.concat(all_results, axis=0)

    # Extract predicted operator fraction
    predicted_operators = total_results.groupby(["model", "normalized_card"])[
        "predicted_operator"].value_counts().reset_index()
    predicted_operators = predicted_operators[predicted_operators["predicted_operator"] == "seq"]
    predicted_operators["count"] = predicted_operators["count"] / len(seq_workloads)

    # Extract optimal operator fraction
    optimal_operators = total_results.groupby(["model", "normalized_card"])[
        "optimal_operator"].value_counts().reset_index()
    optimal_operators = optimal_operators[optimal_operators["optimal_operator"] == "seq"]
    optimal_operators["count"] = optimal_operators["count"] / len(seq_workloads)
    optimal_operators = optimal_operators[optimal_operators["model"] == "Sc. PG10"]
    # For all selectivity values between 0 and 1, set the value to 0 if not existent in dataframe
    for i in range(0, 11):
        if not i / 10 in optimal_operators["normalized_card"].values:
            optimal_operators = optimal_operators._append(
                {"model": "Sc. PG10", "normalized_card": i / 10, "optimal_operator": "seq", "count": 0},
                ignore_index=True)

    # Sort by model_config name
    predicted_operators = predicted_operators.sort_values(by="model", key=lambda x: x.map(
        {model.name.DISPLAY_NAME: i for i, model in enumerate(model_conf)}))
    optimal_operators = optimal_operators.sort_values(by="model", key=lambda x: x.map(
        {model.name.DISPLAY_NAME: i for i, model in enumerate(model_conf)}))

    x = "normalized_card"
    seaborn.lineplot(x=x, y="count", data=optimal_operators, ax=ax, color="black", linewidth=2, zorder=100)
    seaborn.scatterplot(x=x, y="count", data=optimal_operators, ax=ax, color="black", legend=False, marker="o", s=90,
                        label="Optimal Selection", zorder=100)
    seaborn.lineplot(x=x, y="count", hue="model", data=predicted_operators, ax=ax, palette=ColorManager.COLOR_PALETTE,
                     legend=False)
    seaborn.scatterplot(x=x, y="count", hue="model", data=predicted_operators, ax=ax,
                        palette=ColorManager.COLOR_PALETTE, legend=True, markers=markers, style="model", s=90)

    ax.set_ylabel("Ratio of Sequential Scans", fontsize=fontsize)
    ax.set_xlabel("Selectivity", fontsize=fontsize)
    ax.set_ylim(-0.05, 1.05)
    ax.set_xlim(-0.05, 1.05)

    # Add circled letter to the top left corner to disinguish plots
    letter = "A" if model_conf == model_configs else "B"
    ax.annotate(
        letter,
        xy=(0.01, 1.06),
        xycoords='axes fraction',
        fontsize=12,
        ha='center',
        va='center',
        bbox=dict(boxstyle='circle,pad=0.2', edgecolor='black', facecolor='white'))

axs[0].legend(loc='center right',
              bbox_to_anchor=(2.0, -0.4),
              edgecolor='white',
              labelspacing=0.2,
              ncol=4,
              columnspacing=0.8,
              fontsize=fontsize * 0.8)

axs[1].legend().remove()
axs[0].set_title("Using estimated cardinalities", fontsize=fontsize)
axs[1].set_title("Using actual cardinalites", fontsize=fontsize)

fig.align_labels()
plt.subplots_adjust(wspace=0.2)
fig.savefig(path, bbox_inches='tight')
plt.show()

# Retraining Experiments

In [ ]:
label_encoder = LabelEncoder()
path = LocalPaths().data / "plots" / "retraining.pdf"
mosaic = """AAAABB\nAAAACC\nAAAADD"""

seq_workloads, idx_workloads = EvalWorkloads.ScanCostsPercentile.imdb_seq, EvalWorkloads.ScanCostsPercentile.imdb_idx
model_configs =[ScaledPostgresModelConfig(), FlatModelConfig(), E2EModelConfig(),  QueryFormerModelConfig(), ZeroShotModelConfig(), DACEModelConfig()]

balanced_accuracy_scores = []
runtimes = []

for seq_workload, idx_workload in zip(seq_workloads, idx_workloads):
    
    for retraining in [False, True]:
        # Get and rename results
        seq_results = get_model_results(seq_workload, model_configs, retraining).rename(columns={'prediction': 'prediction_seq', 'label': 'label_seq'})
        idx_results = get_model_results(idx_workload, model_configs, retraining).rename(columns={'prediction': 'prediction_idx', 'label': 'label_idx'})
        results = pd.concat([seq_results, idx_results[['prediction_idx', 'label_idx']]], axis=1)
        
        # Remove slightly deviating query labels:
        results['label_idx'] = results.groupby('query_index')['label_idx'].transform('last')
        results['label_seq'] = results.groupby('query_index')['label_seq'].transform('last')
        
        # Assign optimal and predicted operators
        results = results.assign(
            optimal_operator=np.where(results['label_idx'] < results['label_seq'], 'idx', 'seq'),
            predicted_operator=np.where(results['prediction_idx'] < results['prediction_seq'], 'idx', 'seq'))
        
        # Give the index of the query where the label difference between idx and seq is minimal
        min_diff_idx = (results['label_idx'] - results['label_seq']).abs().idxmin()
        max_label = max(results['label_idx'].max(), results['label_seq'].max())
        
        # Print the selectivity of the query where the difference is minimal
        print(seq_workload.get_workload_name())
        
        # Create selected_runtime column
        results['selected_runtime'] = np.where(results['prediction_idx'] < results['prediction_seq'], results['label_idx'], results['label_seq'])
        
        # Scale act_card and calculate accuracy
        results['act_card'] = (results['act_card'] - results['act_card'].min()) / (results['act_card'].max() - results['act_card'].min())
        #results['accuracy'] = (results['optimal_operator'] == results['predicted_operator']).astype(int)
        #results['accuracy'] = balanced_accuracy_score(results['optimal_operator'], results['predicted_operator'])
        results.loc[:, 'predicted_operator_encoded'] = label_encoder.fit_transform(results['predicted_operator'])
        results.loc[:, 'optimal_operator_encoded'] = label_encoder.fit_transform(results['optimal_operator'])
        
        
        for model in model_configs:
            model_results = results[results['model'] == model.name.DISPLAY_NAME]
    
            # Delete rows that contain NaN values (which happens for timeouts)
            model_results = model_results.dropna()
        
            # Calculate balanced accuracy        
            accuracy = balanced_accuracy_score(model_results["optimal_operator"], model_results["predicted_operator"])
            balanced_accuracy_scores.append((seq_workload.get_workload_name(), model.name.DISPLAY_NAME, accuracy, retraining))
            
            # Calculate total runtime
            runtime = model_results["selected_runtime"].sum()
            runtimes.append((seq_workload.get_workload_name(), model.name.DISPLAY_NAME, runtime, retraining))

In [ ]:
scores = pd.DataFrame(balanced_accuracy_scores, columns=["workload", "model", "accuracy", "retraining"])

# Pivot the data to separate 'True' and 'False' retraining values
false_df = scores[scores['retraining'] == False].set_index('model')
true_df = scores[scores['retraining'] == True].set_index('model')
pivot_df = scores.pivot_table(index=['workload', 'model'], columns='retraining', values='accuracy').fillna(0)

# Extract unique workloads and models
workloads = scores['workload'].unique()
models = scores['model'].unique()

# Set the figure and axes for plotting
fig, (ax, ax2) = plt.subplots(1, 2, figsize=(8, 2), width_ratios=[3, 1])

# Set bar width
bar_width = 0.12
# Set the number of workloads and their positions
workload_positions = np.arange(len(workloads))

labels, handles = [], []
# Loop over models to create a stacked bar for each workload
for i, model in enumerate(model_configs):
    # Offsets each model within a workload group
    offset = i * bar_width +0.36
    # Get false and true values for the current model
    false_values = pivot_df.xs(model.name.DISPLAY_NAME, level='model')[False].reindex(workloads)
    true_values = pivot_df.xs(model.name.DISPLAY_NAME, level='model')[True].reindex(workloads)
    
    if isinstance(model, FlatModelConfig):
        color = ColorManager.ALTERNATIVE[5]
    elif isinstance(model, E2EModelConfig):
        color = ColorManager.ALTERNATIVE[3]
    elif isinstance(model, QueryFormerModelConfig):
        color = ColorManager.ALTERNATIVE[0]
    elif isinstance(model, ZeroShotModelConfig):
        color = ColorManager.ALTERNATIVE[6]
    elif isinstance(model, DACEModelConfig):
        color = ColorManager.ALTERNATIVE[1]
    else:
        color ="gray"
        
    ax.bar(workload_positions + offset, true_values, bar_width, bottom=0, label=f'{model} (Retraining True)', color=color, edgecolor="black")
    
    ax.bar(workload_positions + offset, false_values, bar_width, label=f'{model} (Retraining False)',  color=model.color(), edgecolor="black")
    
    if isinstance(model, ScaledPostgresModelConfig) and not isinstance(model, ScaledPostgresModelV16Config):
        labels.append(f'{model.name.DISPLAY_NAME}')
        handles.append(mpatches.Patch(facecolor=model.color(), edgecolor="black"))

    else:
        labels.append(f'{model.name.DISPLAY_NAME}')
        handles.append(mpatches.Patch(facecolor=model.color(), edgecolor="black"))
        labels.append(f'{model.name.DISPLAY_NAME} (fine-tuned)')
        handles.append(mpatches.Patch(facecolor=color, edgecolor="black"))
    
ax.set_xticks(workload_positions + 0.94)
workloads[-1] = workloads[-1].replace("production", "prod")
workloads = [workload.replace("seq.", "").replace(".", "\n") for workload in workloads]
ax.set_xticklabels(workloads, rotation=90, ha='right', fontsize=0.8*fontsize)
ax.set_ylabel('Balanced Accuracy', fontsize=fontsize)
#ax.set_xlabel('Column', fontsize=fontsize)
ax.tick_params(axis='both', which='major', pad=-2)
ax.grid(visible=False, axis="x")


runtimes_df = pd.DataFrame(runtimes, columns=["workload", "model", "runtime", "retraining"])

sort_map = {model.name.DISPLAY_NAME: i for i, model in enumerate(MODEL_CONFIGS)}

# Remove postgres form dataframe
#runtimes_df = runtimes_df[runtimes_df['model'] != "Sc. Postgres"]

# Pivot the data to separate 'True' and 'False' retraining values
false_df = runtimes_df[runtimes_df['retraining'] == False].set_index('model')
true_df = runtimes_df[runtimes_df['retraining'] == True].set_index('model')


# Aggregate the runtimes for each model
false_df = false_df.groupby('model')['runtime'].sum()
true_df = true_df.groupby('model')['runtime'].sum()

# Sort series according to sort map
false_df = false_df.sort_index(key=lambda x: x.map(sort_map))
true_df = true_df.sort_index(key=lambda x: x.map(sort_map))

# Add new column to specify whether the runtime is for retraining or not
false_df = false_df.reset_index()
false_df['retraining'] = False
true_df = true_df.reset_index()
true_df['retraining'] = True

# Remove postgres from dataframe
true_df = true_df[true_df['model'] != "Sc. PG10"]

# Append both dataframes and specify in a new column whether the runtime is for retraining or not
merged_df = pd.concat([false_df, true_df], axis=0)

color_palette = [model.color() for model in model_configs]
sns.barplot(ax=ax2, palette=color_palette, edgecolor='black', data=merged_df, x='model', y='runtime', hue='retraining')
ax2.set_ylabel('Total Runtime (s)', fontsize=fontsize)
ax2.set_xlabel('', fontsize=fontsize)


# Define custom colors for the left and right bars
left_colors = ["gray", FlatModelConfig().color(), E2EModelConfig().color(), QueryFormerModelConfig().color(), ZeroShotModelConfig().color(), DACEModelConfig().color() ]  # Colors for the "Without Retraining" bars (left)
right_colors = [ColorManager.ALTERNATIVE[5], ColorManager.ALTERNATIVE[3], ColorManager.ALTERNATIVE[0], ColorManager.ALTERNATIVE[6], ColorManager.ALTERNATIVE[1]]  # Colors for the "With Retraining" bars (right)

# Get the bars from the plot (each bar is represented as a patch in seaborn)
bars = ax2.patches

# Assign colors based on left/right position in each group
num_models = len(merged_df['model'].unique())

# Iterate through the bars and set colors for left and right bars
for i, bar in enumerate(bars[0:-2]):
    if i < 6:
        bar.set_color(left_colors[i])
    else:
        bar.set_color(right_colors[i - num_models])
    bar.set_edgecolor('black')

#hatches = ['//', '-||', '\\\\', '++', '..', '\\-', 'x', '-']

# Draw hatches
bars[0].set_hatch('//')
bars[1].set_hatch('oo')
bars[2].set_hatch(None)
bars[3].set_hatch('\\\\')
bars[4].set_hatch('...')
bars[5].set_hatch('---')
bars[6].set_hatch('oo')
bars[7].set_hatch(None)
bars[8].set_hatch('\\\\')
bars[9].set_hatch('...')
bars[10].set_hatch('---')
bars = ax.patches
for i in range(6, 12):
    bars[i].set_hatch('//')
for i in range(12, 24):
    bars[i].set_hatch('oo')
for i in range(24, 36):
    bars[i].set_hatch(None)
for i in range(36, 48):
    bars[i].set_hatch('\\\\')
for i in range(48, 60):
    bars[i].set_hatch('...')
for i in range(60, 72):
    bars[i].set_hatch('---')
    
# Add horizontal, dashed line on PG costs
pg_value = merged_df[merged_df['model'] == 'Sc. PG10']['runtime'].values[0]
ax2.axhline(y=pg_value, color='black', linestyle='-', linewidth=1.5)
# Also add a label for the PG costs
ax2.text(2.8, 0.8*pg_value, str(round(pg_value)) + "s", color='black', fontsize=fontsize*0.7, va='center', ha='right', backgroundcolor='white')
    
ax.annotate(
        "A",
        xy=(0.035, 0.9), 
        xycoords='axes fraction', 
        fontsize=10, 
        ha='center', 
        va='center', 
        bbox=dict(boxstyle='circle,pad=0.2', edgecolor='black', facecolor='white'))
ax2.annotate(
        "B",
        xy=(0.1, 0.91), 
        xycoords='axes fraction', 
        fontsize=10, 
        ha='center', 
        va='center', 
        bbox=dict(boxstyle='circle,pad=0.2', edgecolor='black', facecolor='white'))
    
# rotate x-axis labels
#ax2.set_xticklabels(ax2.get_xticklabels(), rotation=45, ha='right', fontsize=fontsize)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right', fontsize=fontsize * 0.8)
ax.tick_params(axis='x', which='major', pad=-7)
ax2.set_xticklabels("")
ax.legend().remove()
handles = ax2.legend(labels=labels, handles=handles, fontsize=fontsize*0.8, ncol=3, labelspacing=0.1,  loc='upper left', bbox_to_anchor=(-4, -0.45), edgecolor='white')

# add hatches to handles
for handle, hatch in zip(handles.legendHandles, ['//', 'oo', 'oo', None, None, '\\\\', '\\\\', '...', '...', '---', '---']):
    handle.set_hatch(hatch)

fig.align_labels()
plt.subplots_adjust(wspace=0.3)
fig.savefig(path, bbox_inches='tight')
plt.show()